In [3]:
from utils import *
import os

In [ ]:
#microdados_operacao
path = ''
d = os.listdir(path)

for file in d:
    file = path+ '/'+ file
    #ler arquivo e setar tipos de dados
    df = pd.read_csv(
        file,
        sep=';',
        encoding='latin1',
        #nrows = 5000,
        dtype= microdados_operacao_dtypes()
    )
    print('csv lido!')

    #renomear colunas
    df = rename_variables(
        df
    )
    print('colunas renomeadas !')

    #formatar datas
    df = format_dates(
        df,
        columns= ['data_emissao', 'data_vencimento']
    )
    print('datas formatadas !')
    
    #criar coluna de ano e mes
    df = extract_create_year_month(
        df,
        data_emissao= 'data_emissao',
        data_vencimento= 'data_vencimento'
    )
    print('ano e mes criados!')

    #criar coluna de plano safras
    df = create_plano_safra_vencimento(
        df,
        mes_col = 'mes_vencimento',
        ano_col = 'ano_vencimento'
    )

    df = create_plano_safra_emissao(
        df,
        mes_col = 'mes',
        ano_col = 'ano'
    )
    print('plano safras criados!')

    #ordenar colunas
    ordem = create_column_order('https://docs.google.com/spreadsheets/d/1CIM_enrWOD6PxahuVdfBAQ8IRFl1HWVD/edit#gid=1419816679')

    df = df[ordem]
    #particionas tabela
    print('colunas ordenadas!')
    
    to_partitions(
        df,
        partition_columns=['ano', 'mes', 'sigla_uf'],
        savepath= r'',
        
    )
    print('tabelas particionadas!')    
    

In [ ]:
#microdados_liberacao

path = r''

#ler arquivo e setar tipos de dados
df = pd.read_csv(
    path,
    sep=';',
    encoding='latin1',
    #nrows = 100000,
    dtype= microdados_liberacao_dtypes()
)
print('csv lido!')

#renomear colunas
df = rename_variables(
    df
)
print('colunas renomeadas !')

#criar colunas de datas
df['data_liberacao'] = df['data_liberacao'].str[:10]
df['ano'] = df['data_liberacao'].str[6:10].astype(str)
df['mes'] = df['data_liberacao'].str[0:2].astype(str)

#formatar datas
values = [
    '1753',
    '2026',          
    '2012',          
    '2106',          
    '0222',          
    '0201',          
    '2000',          
    '2024',          
    '2011',          
    '2109',          
    '2049',          
    '2025',          
    '2501',          
    '2173'
    ]

df = drop_rows_by_value(df, column='ano', values=values)
df = format_dates(df, columns=['data_liberacao'])

ordem = create_column_order('https://docs.google.com/spreadsheets/d/1mBpa7_o-cQeV5AQXkb5etwnkaxbnWiso/edit#gid=1753975247')

df = df[ordem]
#particionas tabela
print('colunas ordenadas!')

to_partitions(
    df,
    partition_columns=['ano', 'mes'],
    savepath= r'',
    
)
print('colunas particionadas!')


In [ ]:
#microdados_saldo

def to_partitions(data: pd.DataFrame, partition_columns: List[str], savepath: str,file_name: str):
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/'
        )
    """

    if isinstance(data, (pd.core.frame.DataFrame)):

        savepath = Path(savepath)

        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            .drop_duplicates(subset=partition_columns)
            .to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}"
                for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()]
                .isin(filter_combination.values())
                .all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)
            # todo: insert file name as name 'data.csv'
            file_filter_save_path = Path(filter_save_path) / f"{file_name}_data.csv"

            # append data to csv
            df_filter.to_csv(
                file_filter_save_path,
                index=False,
                mode="a",
                header=not file_filter_save_path.exists(),
            )
    else:
        raise BaseException("Data need to be a pandas DataFrame")

path = 'C:/Users/gabri/OneDrive/vida_profissional/Projetos/base_dos_dados/br_bcb_sicor/extra/auxiliary_files'
d = os.listdir(path)

for file in d:
    
    complete_file = path+ '/'+ file
    #ler arquivo e setar tipos de dados
    df = pd.read_csv(
        complete_file,
        sep=';',
        encoding='latin1',
       #nrows = 50000,
        dtype= microdados_saldo_dtypes()
    )
    print('csv lido!')
    #renomear colunas
    df = rename_variables(
        df
    )
    print('colunas renomeadas !')

    #ordenar colunas
    ordem = create_column_order('https://docs.google.com/spreadsheets/d/1SWvTxlydEflSED7VevRGL_r8Gh7vTu_6/edit#gid=913462137')

    df = df[ordem]
    #particionas tabela
    print('colunas ordenadas!')

    to_partitions(
        df,
        partition_columns=['ano', 'mes'],
        savepath= r'',
        file_name= file
    )
    print('colunas particionadas!')


In [ ]:
#microdados_saldo

def ufs():
  ufs = ['MA',
         'SP', 
         'ES', 
         'MG', 
         'PR', 
         'SC', 
         'RS', 
         'MS', 
         'GO', 
         'AC', 
         'AL', 
         'AP',
         'AM', 
         'BA', 
         'CE', 
         'DF', 
         'MA', 
         'MT', 
         'PA' 
         'PB', 
         'PE', 
         'PI', 
         'RN', 
         'RO',
         'RR', 
         'SE', 
         'TO'
         ]
  return ufs 

def meses():
  meses = ['1',
           '2', 
           '3',
           '4', 
           '5', 
           '6', 
           '7', 
           '8', 
           '9', 
           '10', 
           '11', 
           '12'
            
           ]
  return meses

def anos():
  anos = [
    '2023',
    '2022',
    '2021',
    '2020',
    '2019',
    '2018',
    '2017',
    '2016',
    '2015',
    '2014',
    '2013'
  ]
  return anos

for ano in anos():
  for mes in meses():
    #2023 doesnt have all months
    try:
      print(f'binding {ano} {mes}')
      #listar arquivos
      files = os.listdir(f'saldo/ano={ano}/mes={mes}')	
      #iterar em cada arquivo
      arquivos = []
      for file in files:
        
        df = pd.read_csv(
          f'saldo/ano={ano}/mes={mes}/{file}',
          sep=',',
          encoding='latin1',
          dtype= microdados_saldo_dtypes()
        )

        arquivos.append(df)
        print(f'binding {ano} {mes} {file}')
        del df
        
        #concatenar lista
        #salvar em csv
        #deletar arquivos
      df = pd.concat(arquivos)
      print(f'files binded: {files}!')
      
      os.makedirs(
        f'saldo_final/ano={ano}/mes={mes}', 
        exist_ok=True
        )
        
      df.to_csv(
        f'saldo_final/ano={ano}/mes={mes}/data.csv',
        na_rep = '',
        encoding = 'latin1',
        index = False
      )
      print(f'file {ano} {mes} {file}  saved')
      del df
    
    except:
      print(f'error binding {ano} {mes}, probably cause it doesnt exist')

